# Note sur les cartes cognitives

V2, après nouvelles enquêtes VBO et reprise du thésaurus.


## Notes sur les données

### A l'usage du producteur

Pour faciliter le traitement automatique, assurer la qualité des données

- apostrophe avant date
- espace/majuscules surperflues : respecter l'orthographe, supprimer les articles inutiles, forme fléchies
  - penser à _un dictionnaire_
- colonnes duppliquées (calcul si besoin)
- colonnes de même nom
- colonnes au nom ambigu
- numéro au delà des utilisés
- pas de mise en forme en utilisant des cellules vides

### Notes techniques

- rester tant que possible sur un `DataFrame`,
  - ne pas mixer / casser la srtucture,
  - ou alors, _once and for all_ e.g., pour la liste de réponses,
- assurer la cohérence de la clef : OK (au 2022-05-31)
- gérer les libellés des questions
- la représentation des durées/ages (`ans`)

## Exploration du dataset


In [ ]:
import operator as op

from datetime import date
from pathlib import Path
from collections import Counter
from statistics import fmean


import numpy as np
import scipy.stats as stats
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

from catk import CA

FILENAME = Path("../input/BDD_2022-06-02.xlsx")
MAX_COLS = 20
PREFIX_MINE = "Mine"
PREFIX_FUTUR = "Futur"
LABEL_MINE = "la mine et le nickel"
LABEL_FUTUR = "la mine dans le futur"
TOP_K_WORDS = 20

cols_mine = [f"{PREFIX_MINE} {i}" for i in range(1, MAX_COLS + 1)]
cols_futur = [f"{PREFIX_FUTUR} {i}" for i in range(1, MAX_COLS + 1)]
cols_censurees = [f"Question {x}" for x in [2, 5.1, 7, 10.1, 11, 14, 14.1]] + ["Commentaire entretien/individu"]


def clean(s: str) -> str:
    return s.strip().lower()


In [ ]:
df_complete = pd.read_excel(FILENAME, sheet_name="BDD", index_col="Numéro")

# ajout des groupes d'age quinquennaux/décénaux
df_complete["GAD"] = 10 * (df_complete["Question 12"] // 10)
df_complete["GAQ"] = 5 * (df_complete["Question 12"] // 5)

print("Liste des colonnes disponibles")
display(df_complete.columns)

cols_resp = [c for c in df_complete.columns if c not in cols_mine + cols_futur + cols_censurees]

df_resp = df_complete[cols_resp]
df_resp.name = "Données personnelles"
df_mine = df_complete[cols_mine].astype("string")
df_mine.name = f"Cartes cognitives '{LABEL_MINE}'"
df_futur = df_complete[cols_futur].astype("string")
df_futur.name = f"Cartes cognitives '{LABEL_FUTUR}'"

print(f"Nombre de répondants {len(df_complete)}")
print(f"Colonnes '{LABEL_MINE}' présentes {set(cols_mine) <= set(df_complete.columns)}")
print(f"Colonnes '{LABEL_FUTUR}' présentes {set(cols_futur) <= set(df_complete.columns)}")


Informations sur les répondants

In [ ]:
# display(df_resp.info())
display(df_resp)


Cartes cognitives 'la mine'

In [ ]:
display(df_mine)


Cartes cognitives 'la mine futur'

In [ ]:
display(df_futur)


Quelques vérifications et indicateurs globaux


Création des dictionnaires de réponses sur les deux enquêtes


In [ ]:
def get_cogmaps(df, cols):
    return {idx: [clean(val) for val in vals.dropna().tolist()] for idx, vals in df[cols].astype("string").iterrows()}


dict_mine = get_cogmaps(df_complete, cols_mine)
dict_futur = get_cogmaps(df_complete, cols_futur)

# display(dict_mine)

for dic, lbl in ((dict_mine, LABEL_MINE), (dict_futur, LABEL_FUTUR)):
    print(f"Cartes '{lbl}'")
    print(f"\tplus longue cartes {max(len(val) for val in dic.values())}")
    print(f"\tlongueur moyenne des cartes {fmean(len(val) for val in dic.values()):.2f}")
    print(f"\tnombre total de mots énoncés {sum(len(val) for val in dic.values())}")
    print(f"\tnombre de mots énoncés différents {len(set(v for val in dic.values() for v in val))}")
    print()


In [ ]:
for dic, lbl in ((dict_mine, LABEL_MINE), (dict_futur, LABEL_FUTUR)):
    words = Counter(w for l in dic.values() for w in l)
    print(f"Cartes '{lbl}' : {len(words)} mots. Top {TOP_K_WORDS}, toutes positions confondues :")
    display(words.most_common(TOP_K_WORDS))


Longueur des cartes et leur histogramme


In [ ]:
def get_n_words(dic, n: int):
    """Mot de la position n"""
    return [vals[n] for vals in dic.values() if n < len(vals)]


print("Répartitions des longueurs des cartes")
sns.histplot([len(vals) for vals in dict_mine.values()], bins=20)
plt.show()
sns.histplot([len(vals) for vals in dict_futur.values()], bins=20)
plt.show()


## Quelques indicateurs sur les profils des répondants

On commence par récupérer les libellés des questions


In [ ]:
df_questions = pd.read_excel("../input/BDD_2022-05-31.xlsx", sheet_name="Questions", parse_dates=False).astype(
    {"Question": "string", "Libellé": "string"}
)
display(df_questions)
print(f"Toutes les questions présentes {set(df_questions['Question']) <= set(df_resp.columns)}")
questions_labels = df_questions.set_index("Question").to_dict()["Libellé"]  # orient="series"


In [ ]:
# print(questions_labels["Question 1"])
# print(questions_labels["Question 4"])
def groupby(questions):
    return df_complete.value_counts(subset=questions, sort=False).rename_axis(
        [questions_labels.get(q, q) for q in questions]
    )


display(groupby(["Question 1", "Question 4"]))
display(groupby(["Question 8.1"]))
display(groupby(["GAD", "Question 8.2"]))


In [ ]:
print("Histogramme des ages des répondants")
sns.histplot(df_resp["Question 12"], bins=20)


## Mots les plus communs pour la carte "la mine"

On va segmenter les données en _populations_ (e.g., "moins de 26 ans", "mineur", etc.) selon les attributs et générer un tableau avec :

- en ligne tous les mots énoncés dans les cartes "la mine"
- en colonne, les populations, avec pour chacune :
  - le nombre de fois où le mot a été énoncé par les membres de cette population
  - le nombre de fois "théorique" où le mot serait annoncé si il y avait indépendance des énonciations entre les populations
  - le % de fois où un membre de la population a énoncé le mot
- pour chaque groupe, on calcule ensuite un test du $\chi^2$ pour évaluer l'écart à l'indépendance


In [ ]:
LABEL_TOTAL = "Total"
CLEF_TOTAL = (
    "*",
    "*",
    LABEL_TOTAL,
)
all_words_mine = Counter(w for l in dict_mine.values() for w in l)
all_words_futur = Counter(w for l in dict_futur.values() for w in l)

df_population = pd.DataFrame(index=list(all_words_mine.keys()))
df_population[CLEF_TOTAL] = all_words_mine.values()
THRESHOLD = 20
df_population = df_population[df_population[CLEF_TOTAL] >= THRESHOLD]
df_population.sort_values(CLEF_TOTAL, key=op.neg, inplace=True)


def pretty_pvals(x):
    if x < 1e-4:
        return "****"
    if x < 1e-3:
        return "***"
    if x < 1e-2:
        return "**"
    if x < 5e-2:
        return "*"
    return ""


def add_population(filtres, name):
    """Ajout de colonnes sur une population segmentée"""
    observed = []
    expected = []
    chi2 = stats.chi2(len(filtres) - 1)
    for filtre, val in filtres:
        cg = get_cogmaps(df_mine[filtre], cols_mine)
        c = Counter(w for l in cg.values() for w in l)
        print(f"Ajout de la colonne '{name}, {val}', {len(cg)} répondants, {c.total()} énonciations")

        key = (name, val, "Nb.")
        df_population[key] = pd.Series(c)
        df_population[key].fillna(0, inplace=True)
        df_population[key] = df_population[key].astype(int)
        # df_population[f"{name} (% des mots)"] = round(100 * pd.Series(c) / c.total(),2)
        # ici en % ** des répondants**
        df_population[(name, val, "Nb. th.")] = round(df_population[CLEF_TOTAL] * len(cg) / len(df_mine), 2)
        exp = df_population[CLEF_TOTAL].to_numpy() * len(cg) / len(df_mine)
        # df_population[(name, val, "(%)")] = round(100 * pd.Series(c) / len(cg), 2)

        observed.append(df_population[key].to_numpy().reshape(-1, 1))
        # expected.append(df_population[(name, val, "Nb. th.")].to_numpy().reshape(-1, 1))
        expected.append(exp.reshape(-1, 1))

    obs, exp = np.concatenate(observed, axis=1), np.concatenate(expected, axis=1)
    # df_population[(name, "Chi²", "Value")] = np.sum((obs-exp)**2/exp, axis=1)
    chis = np.sum((obs - exp) ** 2 / exp, axis=1)
    # df_population[(name, "Chi²", "p-value")] = chi2.sf(df_population[(name, "Chi²", "Value")])
    p_vals = chi2.sf(chis)
    # df_population[(name, "Chi²", "Significatif")] = [pretty_pvals(v) for v in  df_population[(name, "Chi²", "p-value")]]
    df_population[(name, "Chi²", "Significatif")] = [pretty_pvals(v) for v in p_vals]


def generate_all_filters(col):
    values = df_resp[col].dropna().unique()
    return [(df_resp[col] == value, value) for value in values]


moins_de_26_ans = df_resp["Question 12"] < 26
add_population([(moins_de_26_ans, "Oui"), (~moins_de_26_ans, "Non")], "Age < 26")
add_population(generate_all_filters("Question 13"), "Genre")
# add_population(generate_all_filters("GAD"), "Groupe d'age décénal")
add_population(generate_all_filters("Question 1"), "Travaille dans la mine")
add_population(generate_all_filters("Question 4"), "Famille dans la mine")
add_population(generate_all_filters("Question 5"), "Toujours vécu en NC")
# add_population(generate_all_filters("Question 8.1"), "Commune de résidence")
# pour ça, plutôt faire une AFC
add_population(generate_all_filters("Question 8.2"), "Commune minière")
# df_population=df_population.copy()


# nettoyage final
df_population.fillna(0.0, inplace=True)
df_population.columns = pd.MultiIndex.from_tuples(df_population.columns)
df_population.columns.set_names(["Variable", "Catégorie", "Statistique"], inplace=True)

# THRESHOLD = 10
# display(df_population[df_population[CLEF_TOTAL] >= THRESHOLD])
display(df_population)
# display(df_population[df_population[("Age < 26", "Chi²", "Significatif")].str.startswith("**")])


In [ ]:
today = date.today().strftime("%Y-%m-%d")
output = Path("../output_v2")
output.mkdir(parents=True, exist_ok=True)
df_population.to_excel(output / f"analyse_par_population-{today}.xlsx")


## Complétude du thésaurus


In [ ]:
df_thesaurus = pd.read_excel(FILENAME, sheet_name="Thésaurus_2")
concepts = df_thesaurus.columns.to_list()


thesaurus = {
    clean(key): [clean(val) for val in vals if val is not np.nan]
    for key, vals in df_thesaurus.to_dict(orient="list").items()
}

print(f"Nombre de concepts dans le thésaurus {len(thesaurus)}")
print(f"Nombre de mots classés dans le thésaurus {sum(len(vals) for vals in thesaurus.values())}")


In [ ]:
all_words_thesaurus = [val for vals in thesaurus.values() for val in vals]
print(f"Nombre de mots du thésaurus = {[len(vals) for vals in thesaurus.values()]}")
print(f"Nombre total de mots du thésaurus = {len(all_words_thesaurus)}")
print(f"Nombre de mots du thésaurus sans doublons = {len(set(all_words_thesaurus))}")
print(f"Doublons dans le thésaurus : {len(all_words_thesaurus) != len(set(all_words_thesaurus))}")


In [ ]:
occ_mutiples = {
    mot: [concept for concept, vals in thesaurus.items() if mot in vals]
    for mot, occ in Counter(all_words_thesaurus).items()
    if occ > 1
}

print(f"{len(occ_mutiples)} mots avec occurrences multiples dans le thésaurus, pour chaque, la liste des concepts où il apparait")
display(occ_mutiples)


In [ ]:
mots_mine_sans_concept = set(all_words_mine) - set(all_words_thesaurus)
print(f"Il y a {len(mots_mine_sans_concept)} mots des cartes '{LABEL_MINE}' qui ne sont pas dans le thésaurus.")
print("On donne les mots et les identifiants des réponses où il apparaissent (<=404 ~ Benoit) ")
mots_mine_sans_concept_pos = {
    mot: [num for num, mots in dict_mine.items() if mot in mots] for mot in mots_mine_sans_concept
}
display(mots_mine_sans_concept_pos)


In [ ]:
mots_futur_sans_concept = set(all_words_futur) - set(all_words_thesaurus)
print(f"Il y a {len(mots_futur_sans_concept)} mots des cartes '{LABEL_FUTUR}' qui ne sont pas dans le thésaurus :")
print("On donne les mots et les identifiants des réponses où il apparaissent (<=404 ~ Benoit) ")
mots_futur_sans_concept_pos = {
    mot: [num for num, mots in dict_futur.items() if mot in mots] for mot in mots_futur_sans_concept
}
display(mots_futur_sans_concept_pos)


In [ ]:
mots_thesaurus_sans_enonciation = set(all_words_thesaurus) - set(all_words_mine) - set(all_words_futur)
print(f"Il y a {len(mots_thesaurus_sans_enonciation)} mots du thésaurus qui n'apparaissent dans aucune carte")
display(mots_thesaurus_sans_enonciation)


## Analyse des correspondances


In [ ]:
ROW_THRESHOLD = 20
COL_THRESHOLD = 100


def gen_ca_dataset(attribut):

    cats = df_complete[attribut].dropna().unique()
    cats_filters = {value: (df_resp[attribut] == value) for value in cats}

    # gad_filters[80.0].index[gad_filters[80.0]].to_list() # to_numpy().nonzero()

    df = pd.DataFrame(index=all_words_mine)
    # dict_mine
    # gad_filters.keys()
    for group in sorted(cats_filters.keys()):
        ids = cats_filters[group].index[cats_filters[group]].to_list()
        cogs = Counter(mot for num, mots in dict_mine.items() if num in ids for mot in mots)
        # print(f"Groupe {group} :  {len(ids)} individus, {len(cogs)} mots énoncés")
        df[group] = pd.Series(cogs)

    df = df.fillna(0).astype(int)
    df.columns.name = questions_labels.get(attribut, attribut)
    df.index.name = "Mot énoncé"
    # df_gad

    row_margin = df.sum(axis=1)
    col_margin = df.sum(axis=0)

    # les mots énoncés au moins 20x pour les classes qui énoncent au moins 100 mots
    return df.loc[row_margin > ROW_THRESHOLD, col_margin > COL_THRESHOLD]


# gen_ca_dataset("Question 8.2")

In [ ]:
sns.set_theme(style="whitegrid", font_scale=1.10, rc={"figure.figsize": (16, 12)})
ca = CA(42)

for attribut in ["GAD", "Question 8.1", "Question 8.2"]:
    ref = gen_ca_dataset(attribut)
    display(ref.sort_index())
    ca.fit(ref)
    # ca.plot(coords=("principal", "standard"))
    # plt.show()
    # ca.plot(coords=("standard", "principal"))
    # plt.show()
    ca.plot(coords=("principal", "principal"), legend=None)
    plt.show()
